# Hangman

## Game rule:

When a player plays Hangman, the program randomly selects a secrete word from the word dictionary. Each time, the player guess a letter, if the letter is in the secrete word, the system will return the location of that letter; if it is not, the player lose one chance. In total, the player gets 6 chances of incorrect guesses. In the end, the player either win the game by guessing the word within 6 incorrect guesses or lose.


The dictionary contains 370099 words in total.

## Strategy:

#### Abstract: 
Basic idea is to mimic the n-gram model in text mining, but here instead of paritioning sentence into words, I partition the word into n contiguous letters. Then I partition the word to guess in the same way and match the n-gram model. Finally use simple Bayesian probability to decide which letter to guess next. The algorithm gets 50% success rate for 6 chances.

#### Step 1:
For the word in given dictionary, we partition each word into bigram, trigram, fourgram and fivegram. For example, for the word 'apple' of bigram model, the word is partitioned into 'ap', 'pp', 'pl', 'le'. For the given training data, we loop over all the word and four n-gram models, and save the frequency distributions of 4 n-gram models into dictionary. So the dictionary of bigram looks like {'aa': 100, 'ab':100,... ,'zz':0}, etc.
This step is set aside the class Hangman API to avoid repetitive calculation of frequency. Once built, values can be directly used according to keys.

#### Step 2:
For the first guess when the word given is composed of all underlines like this '_ _ _ _ _'. We guess it according to the frequency of alphabets of words with the matched length. For example, the given word is like the above one, we first find all words of length 5 in the training data, then calculate the frequency of each alphabet for all these matched word. Then we make the first guess on the letter which has the highest frequency, which is typically vowels like 'e', 'i', etc. But for words of different lengths, the first guess still varies.

#### Step 3:
Once, we make the first right guess. We get the word like '_ e _ _ _'. Then we try to get more information based on the given information, i.e., the given 'e'. To maximize P(letter_i | 'e'), by Bayesian, we need to maximize P(letter_i ,'e'), so I search the dictionary of bigram model, retrieve all keys match regular expression of '.e' and 'e.', and save the corresponding values of that key to the letter_i. For example, if 'le' has frequency 12000 and 'el' has frequency 5000, then we frequency of letter l is 12000 + 5000 = 17000. Then for each alphabet a-z, we calculate the frequency, and make the guess on the one with the highest frequency.


Then for 'l e _ _ _', we apply trigram model because it contains the pattern 'le_', which is to maximize P(letter_i | 'le'). We search keys in trigram dictionary which matches the pattern 'le.', and save the frequency of matched last letter. For trigram model, we only consider 3 scenerios, i.e., 'ab_', 'a_b', '_ab', which is two letters within 3 are known and one is blank. For fourgram, we also consider the pattern with only one blank, which are '_abc' , 'a_bc' , 'ab_c' , 'abc_'. For fivegram model, we still consider only one blank, which are '_abcd', 'a_bcd', 'ab_cd' , 'abc_d', 'abcd_'.

#### Step 4:
Final decision rule is that once the given word matches fivegram pattern, which is it only contains one blank for length of 5, we decide simply by frequency given by fivegram model; if no, try the fourgram, then trigram, finally bigram. 


Reason: 1. Magnitude of frequency of different grams are different. For bigram, we may get 20000 for 'in' but for fivegram like 'inter' we may get only like 500, for others maybe fewer. But we can still combine all gram models together by using final_frequency = c1 * bigram_freq + c2 * trigram_freq + c3 * fourgram_freq + c4 * fivegram_freq, and then tune coefficients.
2. Higher gram means more information given. It is more reasonable to guess 'r' by using 'inte_' than guessing 'r' by simply using 'e_'.



#### Assumption:
1. Distributions of alphabets frequency for words in training data are the same as those in the test data. It means, for example, for bigram frequency distribution {'aa': 100, 'ab':100,... 'zz':0}, the distributions are roughly the same.
2. Higher gram models contain more information than lower grams.



In [2]:
import random
import string
import re

In [5]:
dictionary_location = "words_dict.txt"
text_file = open(dictionary_location,"r")
full_dictionary = text_file.read().splitlines()
text_file.close()

In [6]:
len(full_dictionary)

370099

In [8]:
full_dictionary[:5]

['a', 'aa', 'aaa', 'aah', 'aahed']

In [9]:
# form bigram, trigram, four_gram, five_gram frequency distribution for all words in training

def grams_freq(dictionary):
    freq_2 = {}
    freq_3 = {}
    freq_4 = {}
    freq_5 = {}

    for word in dictionary:
        if len(word) >= 2:
            for i in range(len(word) - 1):
                tmp = word[i : (i + 2)]
                if tmp not in freq_2:
                    freq_2[tmp] = 1
                else:
                    freq_2[tmp] += 1
        if len(word) >= 3:
            for i in range(len(word) - 2):
                tmp = word[i : (i + 3)]
                if tmp not in freq_3:
                    freq_3[tmp] = 1
                else:
                    freq_3[tmp] += 1
        if len(word) >= 4:
            for i in range(len(word) - 3):
                tmp = word[i : (i + 4)]
                if tmp not in freq_4:
                    freq_4[tmp] = 1
                else:
                    freq_4[tmp] += 1
        if len(word) >= 5:
            for i in range(len(word) - 4):
                tmp = word[i : (i + 5)]
                if tmp not in freq_5:
                    freq_5[tmp] = 1
                else:
                    freq_5[tmp] += 1
    return freq_2 , freq_3 , freq_4 , freq_5

freq_2 , freq_3 , freq_4 , freq_5 = grams_freq(full_dictionary)


In [17]:
dict(list(freq_2.items())[0:5])

{'aa': 255, 'ah': 1449, 'he': 18798, 'ed': 30877, 'hi': 14800}

In [16]:
dict(list(freq_5.items())[0:5])

{'aahed': 1, 'aahin': 1, 'ahing': 4, 'aalii': 2, 'aliis': 1}

In [27]:
class Hangman(object):
    def __init__(self):
        self.guessed_letters = []

    ## unigram model may only be used once for the first guess till correct, because for afterward guess, unigram model
    ## character frequency may shift others frequency like bigram, trigram a lot. (not sure, to be tested)

    # dictionary is the training data, and current_word is the current word to guess like 'a__le' (without stripes).
    def unigram(self, dictionary , current_word):
        len_word = len(current_word)
        new_dictionary = []
        for word in dictionary:
            if len(word) == len_word:
                new_dictionary.append(word)
            
        # initialize the unigram alphabet frequency dictionary with 0 for each character
        unigram_freq = dict.fromkeys(string.ascii_lowercase, 0)
        for word in new_dictionary:
            # count each distinct charater once for each word, that is, duplicate characters are counted once in each word
            for char in list(set(word)):
                unigram_freq[char] += 1   
        return unigram_freq
    

    # form bigram, trigram, four_gram, five_gram frequency distribution for all words in training
    def grams_freq(self,dictionary):
        freq_2 = {}
        freq_3 = {}
        freq_4 = {}
        freq_5 = {}

        for word in dictionary:
            if len(word) >= 2:
                for i in range(len(word) - 1):
                    tmp = word[i : (i + 2)]
                    if tmp not in freq_2:
                        freq_2[tmp] = 1
                    else:
                        freq_2[tmp] += 1
            if len(word) >= 3:
                for i in range(len(word) - 2):
                    tmp = word[i : (i + 3)]
                    if tmp not in freq_3:
                        freq_3[tmp] = 1
                    else:
                        freq_3[tmp] += 1
            if len(word) >= 4:
                for i in range(len(word) - 3):
                    tmp = word[i : (i + 4)]
                    if tmp not in freq_4:
                        freq_4[tmp] = 1
                    else:
                        freq_4[tmp] += 1
            if len(word) >= 5:
                for i in range(len(word) - 4):
                    tmp = word[i : (i + 5)]
                    if tmp not in freq_5:
                        freq_5[tmp] = 1
                    else:
                        freq_5[tmp] += 1
        return freq_2 , freq_3 , freq_4 , freq_5

              

    
    # Consider only 2 scenerios for bigram model, '_a' and 'a_' , that is, to form the bigram frequency table for unknown
    # characters next to know characters, suppose there is some known character for current word
    # Suppose for bigram model, there is no difference for frequency distribution of different lengths of word
    def bigram(self, freq_2 , current_word):
        bigram_freq = dict.fromkeys(string.ascii_letters, 0)
        
        if len(current_word) >= 2:
            for i in range(len(current_word) - 1):
                bigram_tmp = current_word[i : (i + 2)]

                # pattern '_a'
                if re.match('_.',bigram_tmp) and bigram_tmp != '__':
                    pattern = '.' + bigram_tmp[1]
                    for key in freq_2.keys():
                        if re.match(pattern , key):
                            bigram_freq[key[0]] += freq_2[key]
                # pattern 'a_'
                elif re.match('._' , bigram_tmp) and bigram_tmp != '__':
                    pattern = bigram_tmp[0] + '.' 
                    for key in freq_2.keys():
                        if re.match(pattern , key):
                            bigram_freq[key[1]] += freq_2[key]
        return bigram_freq



    def trigram(self , freq_3 , current_word):
        trigram_freq = dict.fromkeys(string.ascii_letters, 0)
        
        if len(current_word) >= 3:
            for i in range(len(current_word) - 2):
                trigram_tmp = current_word[i : (i + 3)]
                
                # for pattern '_ab', record the alphabet frequency for the first character
                if re.match('_[a-z]{2}',trigram_tmp):
                    pattern = '.' + trigram_tmp[1 : 2]
                    for key in freq_3.keys():
                        if re.match(pattern , key):
                            trigram_freq[key[0]] += freq_3[key]

                # for pattern 'a_b'
                elif re.match('[a-z]_[a-z]' , trigram_tmp):
                    pattern = trigram_tmp[0] + '.' + trigram_tmp[2]
                    for key in freq_3.keys():
                        if re.match(pattern , key):
                            trigram_freq[key[1]] += freq_3[key]

                # for pattern 'ab_'
                elif re.match('[a-z]{2}_' , trigram_tmp):
                    pattern = trigram_tmp[0 : 2] + '.' 
                    for key in freq_3.keys():
                        if re.match(pattern , key):
                            trigram_freq[key[2]] += freq_3[key]
                            
        return trigram_freq
    


    def fourgram(self , freq_4 , current_word):
        fourgram_freq = dict.fromkeys(string.ascii_letters, 0)
        
        if len(current_word) >= 4:
            for i in range(len(current_word) - 3):
                fourgram_tmp = current_word[i : (i + 4)]

                # pattern '_abc'
                if re.match('_[a-z]{3}',fourgram_tmp):
                    pattern = '.' + fourgram_tmp[1 : 4]
                    for key in freq_4.keys():
                        if re.match(pattern , key):
                            fourgram_freq[key[0]] += freq_4[key]

                # pattern 'a_bc'           
                elif re.match('[a-z]_[a-z]{2}' , fourgram_tmp):
                    pattern = fourgram_tmp[0] + '.' + fourgram_tmp[2 : 4] 
                    for key in freq_4.keys():
                        if re.match(pattern , key):
                            fourgram_freq[key[1]] += freq_4[key]
                
                # pattern 'ab_c'
                elif re.match('[a-z]{2}_[a-z]' , fourgram_tmp):
                    pattern = fourgram_tmp[0 : 2] + '.' + fourgram_tmp[3]
                    for key in freq_4.keys():
                        if re.match(pattern , key):
                            fourgram_freq[key[2]] += freq_4[key]
                
                # pattern 'abc_'
                elif re.match('[a-z]{3}_' , fourgram_tmp):
                    pattern = fourgram_tmp[0 : 3] + '.' 
                    for key in freq_4.keys():
                        if re.match(pattern , key):
                            fourgram_freq[key[3]] += freq_4[key]
                             
        return fourgram_freq
    



    def fivegram(self, freq_5 , current_word):
        fivegram_freq = dict.fromkeys(string.ascii_letters, 0)

        if len(current_word) >=5 :
            for i in range(len(current_word) - 4):
                fivegram_tmp = current_word[i : (i + 5)]

                # pattern '_abcd'
                if re.match('_[a-z]{4}' , fivegram_tmp):
                    pattern = '.' + fivegram_tmp[1 : 5]
                    for key in freq_5.keys():
                        if re.match(pattern , key):
                            fivegram_freq[key[0]] += freq_5[key]

                # pattern 'a_bcd'
                elif re.match('[a-z]_[a-z]{3}' , fivegram_tmp):
                    pattern = fivegram_tmp[0] + '.' + fivegram_tmp[2 : 5]
                    for key in freq_5.keys():
                        if re.match(pattern , key):
                            fivegram_freq[key[1]] += freq_5[key]
                
                # pattern 'ab_cd'
                elif re.match('[a-z]{2}_[a-z]{2}' , fivegram_tmp):
                    pattern = fivegram_tmp[0 : 2] + '.' + fivegram_tmp[3 : 5]
                    for key in freq_5.keys():
                        if re.match(pattern , key):
                            fivegram_freq[key[2]] += freq_5[key]

                # pattern 'abc_d'
                elif re.match('[a-z]{3}_[a-z]' , fivegram_tmp):
                    pattern = fivegram_tmp[0 : 3] + '.' + fivegram_tmp[4]
                    for key in freq_5.keys():
                        if re.match(pattern , key):
                            fivegram_freq[key[3]] += freq_5[key]
                 
                # pattern 'abcd_'
                elif re.match('[a-z]{4}_' , fivegram_tmp):
                    pattern = fivegram_tmp[0 : 4] + '.' 
                    for key in freq_5.keys():
                        if re.match(pattern , key):
                            fivegram_freq[key[4]] += freq_5[key]
                
        
        return fivegram_freq


                


    def guess(self, word): # word input example: "_pp_e"
        
        # clean the word so that we strip away the space characters
        clean_word = word
        len_word = len(clean_word)
    
        # for the first guess when all are blank
        if clean_word == '_' * len_word:
            self.freq_f = self.unigram(self.full_dictionary , clean_word)

        else:
            self.freq_f  = dict.fromkeys(string.ascii_letters, 0)
            bigram_freq = self.bigram(freq_2 , clean_word)
            trigram_freq = self.trigram(freq_3 , clean_word)
            fourgram_freq = self.fourgram(freq_4 , clean_word)
            fivegram_freq = self.fivegram(freq_5 , clean_word)

            if any(value != 0 for value in fivegram_freq.values()):
                for key in self.freq_f.keys():
                    self.freq_f[key] = fivegram_freq[key]
            elif any(value != 0 for value in fourgram_freq.values()):
                for key in self.freq_f.keys():
                    self.freq_f[key] = fourgram_freq[key]
            elif any(value != 0 for value in trigram_freq.values()):
                for key in self.freq_f.keys():
                    self.freq_f[key] = trigram_freq[key]
            else:
                for key in self.freq_f.keys():
                    self.freq_f[key] = bigram_freq[key]

        
            # add the alphabet frequency for 4 gram models and store in the final frequency table self.freq_f
            # for key in self.freq_f.keys():
            #     self.freq_f[key] = bigram_freq[key] + trigram_freq[key] + fourgram_freq[key] + fivegram_freq[key]
        

        # sort the keys of self.freq_f in descending order
        sorted_letter_count = sorted(self.freq_f, key = self.freq_f.get, reverse = True)    

        # pick the letter with largest frequency and meanwhile not be guessed before
        for letter in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        
        return guess_letter
        
        
    
    def start_game(self):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        #self.current_dictionary = self.full_dictionary
        
        # randomly select a word to guess from the full dictionary
        word = random.choice(full_dictionary)
        return_word = "_" * len(word)
        
        tries_remains = 6
        print('New game start! No. of tries remains:', tries_remains)
        
        while tries_remains > 0 :            
            guess_letter = self.guess(word)
        
            self.guessed_letters.append(guess_letter)
            
            if guess_letter in word:
                ind = [i for i, a in enumerate(word) if a == guess_letter]
                for i in range(len(ind)):
                    return_word[i] = guess_letter
                if '_' not in return_word:
                    print('SUCCESS!! The word is: ' + word)
                    break
                else:
                    print('Correct!', 'Now the word is: ' + return_word + '. No. of tries remains:' , tries_remains)
            else:
                tries_remains -= 1
                print('Wrong!' , 'The word is still:' + return_word +'. No. of tries remains:' , tries_remains)
            
            
        if '_' in return_word:
            print('You lose the game. The word to guess is: ' + word)
            
    
    
### need strip to chekc the length

In [28]:
Hangman().start_game()

New game start! No. of tries remains: 6
Wrong! The word is still:______________. No. of tries remains: 5
Wrong! The word is still:______________. No. of tries remains: 4


TypeError: 'str' object does not support item assignment